In [1]:
pip install dnspython

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import dns.resolver

INPUT_FILE = "sms_mms_email_domains-ALL"
OUTPUT_FILE = "sms_mms_email_domains-WORKING"

def check_mx_record(domain: str) -> bool:
    try:
        answers = dns.resolver.resolve(domain, "MX")
        return bool(answers)
    except (
        dns.resolver.NoNameservers,
        dns.resolver.NoAnswer,
        dns.resolver.NXDOMAIN,
        dns.resolver.Timeout,
    ):
        return False
    except Exception:
        # Catch-all so a single weird domain doesn't stop the run
        return False

def load_domains(path: str) -> list[str]:
    domains: list[str] = []
    seen = set()

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            raw = line.strip()
            if not raw or raw.startswith("#"):
                continue

            # Allow "domain, ..." or "domain some comment" by taking first token
            d = raw.split()[0].strip().strip(",").lower().rstrip(".")

            if d and d not in seen:
                seen.add(d)
                domains.append(d)

    return domains

def write_domains(domains: list[str], path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        for d in domains:
            f.write(d + "\n")

def main() -> None:
    domains = load_domains(INPUT_FILE)

    working: list[str] = []
    for d in domains:
        if check_mx_record(d):
            working.append(d)

    write_domains(working, OUTPUT_FILE)
    print(f"Checked {len(domains)} domains, wrote {len(working)} working MX domains to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

Checked 294 domains, wrote 168 working MX domains to sms_mms_email_domains-WORKING
